In [ ]:
#Cypher code
from random import randrange, shuffle
import numpy as np
import math

ctext = 'sopxrqprphastzopfinbpbrupscprmasnprcbpoz pfnifjqpx a pqoasjsctpozsaoc'
def decode(text, trials=10, swaps=50000):
    print ("cleaning text")
    # Remove any spaces in the cipher text.
    #cipherText = text.replace(" ", "")
    # Ensure the cipher text is lower case.
    cipherText = text.lower()
    # Find the best key to unlock the cipher text.
    print ("  * Finding a good key to unlock the file...\n")
    key = search(cipherText, trials, swaps)
    # Transform the cipher text into plain text.
    plainText = transform(cipherText, key)
    # Print the plain text.
    print ("  * File decoded:\n")
    print (key)
    print(plainText)
    
def search(text,trials,swaps):
    # Set up default key.
    key = ['r', 'k', 'f', 'b', ' ', 'h', 't', 'z', 's', 'g', 'j', 'n', 'd', 'c', 'i', 'm', 'y', 'a', 'q', 'o', 'e', 'v', 'x', 'w', 'u', 'l', 'p']
    #set up transition rate dictionary
    scoring_params = create_scoring_params_dict('warandpeace.txt')
    # Set bestScore to zero.
    bestScore = 0
    # For each trial...
    for i in range(0, trials):
        # Shuffle key into a random permutation,
        #shuffle(key)
        bestTrialScore = get_cipher_score(text,key,scoring_params)
        # For each swap...
        for j in range(0, swaps):
            # Swap two letters in the key and call this newKey,
            newKey = swap(key[:])
            # Measure how good newKey is and call this newScore,
            newScore = get_cipher_score(text,newKey,scoring_params)
            # If newScore is better than bestTrialScore...
            acceptance_probability = min(1,math.exp(newScore-bestTrialScore))
            if newScore > bestTrialScore:
                # ...then make key equal to newKey...
                key = newKey[:]
                # ...and make bestTrialScore equal to newScore.
                bestTrialScore = newScore
            # But if newScore is less than bestTrialScore...
            elif newScore <= bestTrialScore:
                unif = np.random.uniform(0,1)
                # then accept anyway if random draw is less than ratio
                if  unif < acceptance_probability: #newScore/bestTrialScore:
                    # ...then accept newKey anyway.
                    key = newKey[:]
            if (j%5000==0):
                print ("iter " + str(j) + ":" + transform(text, newKey))
        # If the best score for that trial was better than the all time best score...
        if bestTrialScore > bestScore:
            # ...then make key the bestKey... 
            bestKey = key[:]
            # ...and make bestScore equal to bestTrialScore.
            bestScore = bestTrialScore
            print(bestScore)
            print(transform(text, bestKey))
    # Return the best key found in this random walk.
    return bestKey
'''
Input is a long text
Output is a dictionary of counts for transitions
{'ab'=35,'ac'=366}
'''
def create_scoring_params_dict(longtext_path):
    scoring_params = {}
    alphabet_list = list('abcdefghijklmnopqrstuvwxyz ')
    with open(longtext_path) as fp:
        for line in fp:
            data = list(line.strip())
            for i in range(len(data)-2):
                alpha_i = data[i].lower()
                alpha_j = data[i+1].lower()
                alpha_k = data[i+2].lower()
                if alpha_i not in alphabet_list: #and alpha_i != " ":
                    alpha_i = " "
                if alpha_j not in alphabet_list: #and alpha_j != " ":
                    alpha_j = " "
                if alpha_k not in alphabet_list:# and alpha_j != " ":
                    alpha_k = " "
                key = alpha_i+alpha_j
                if key in scoring_params:
                    scoring_params[key]+=1
                else:
                    scoring_params[key]=1
                key = alpha_i+alpha_j+alpha_k
                if key in scoring_params:
                    scoring_params[key]+=1
                else:
                    scoring_params[key]=1
    return scoring_params

# This function takes as input a text and creates scoring_params dict which contains the 
# number of time each pair of alphabet appears together
# Ex. {'AB':234,'TH':2343,'CD':23 ..}

def score_params_on_cipher(text):
    scoring_params = {}
    alphabet_list = list('abcdefghijklmnopqrstuvwxyz ')
    data = list(text.strip())
    for i in range(len(data)-2):
        alpha_i =data[i].lower()
        alpha_j = data[i+1].lower()
        alpha_k = data[i+2].lower()
        if alpha_i not in alphabet_list:# and alpha_i != " ":
            alpha_i = " "
        if alpha_j not in alphabet_list:# and alpha_j != " ":
            alpha_j = " "
        if alpha_k not in alphabet_list:# and alpha_j != " ":
            alpha_k = " "
        key = alpha_i+alpha_j+alpha_k
        if key in scoring_params:
            scoring_params[key]+=1
        else:
            scoring_params[key]=1
        key = alpha_i+alpha_j
        if key in scoring_params:
            scoring_params[key]+=1
        else:
            scoring_params[key]=1
    return scoring_params
# This function takes the text to be decrypted and a cipher to score the cipher.
# This function returns the log(score) metric

def get_cipher_score(text,key,scoring_params):
    #cipher_dict = create_cipher_dict(cipher)
    decrypted_text = transform(text, key)
    scored_f = score_params_on_cipher(decrypted_text)
    cipher_score = 0
    for k,v in scored_f.items():
        if k in scoring_params:
            cipher_score += v*math.log(scoring_params[k])
    return cipher_score
'''
def score(key, cipherText,scoring_params):
    # Transform the text using the key.
    candidate = transform(cipherText, key)
    # Set initial score to zero.
    scr = 0
    # For each item in the dictionary of common words and bigrams...    
    for item in scoring_params.keys():
        # ...count the number of times the item occurs, multiply by its frequency
        # score = sum [ f(b1,b2)*log(R(b1,b2)) ]
        scr = scr + candidate.count(item)* math.log(scoring_params[item])
    # Return the overall score.
    return scr
'''
def swap(key):
    # Pick a random number between 0 and 25.
    i = randrange(0, 27)
    # Now pick another random number between 0 and 25.
    j = randrange(0, 27)
    # If and while the two numbers happen to be the same...
    while i == j:
        # ...keep trying to pick a second random number that's not equal to the first.
        j = randrange(0, 27)
    # Swap the two items in the key that are indexed by these random numbers.
    key[i], key[j] = key[j], key[i]
    # Return the new key.
    return key

def transform(cipherText, key):
    #"".join(list) returns elements in list as string
    #needed for table="".maketrans("key","replace")
    table = str.maketrans("".join(key), 'abcdefghijklmnopqrstuvwxyz ')
    return cipherText.translate(table)

print(decode(ctext))


cleaning text
  * Finding a good key to unlock the file...

iter 0:it kas a fright cold day in april and the clocws kere striwing thirtn
iter 5000:it was a oright cpld day in afril and the clpcks were striking thirtn
iter 10000:it was a prifht cold day in abril and the clocks were strikinf thirtn
iter 15000:it was a pright cold day in afril and the clocks were striking thirtn
iter 20000:it was a fright cold day im april amd the clocks were strikimg thirtm
iter 25000:it wae a pright cold day in afril and ths clocke wsrs etriking thirtn
iter 30000:it was a pright cold day in afril and the clocbs were stribing thirtn
iter 35000:it was a prigft cold day in ahril and tfe clocks were striking tfirtn
iter 40000:it was a fright colx xay in april anx the clocks were striking thirtn
iter 45000:it was a pright cold day im afril amd the clocks were strikimg thirtm
839.3648295775083
it was a pright cold day in afril and the clocks were striking thirtn
iter 0:itmwasmamprightmcoldmdayminmafrilmandmth

In [30]:
'''
Input is a long text
Output is a dictionary of counts for transitions
{'ab'=35,'ac'=366}
'''
def create_scoring_params_dict(longtext_path):
    scoring_params = {}
    alphabet_list = list('abcdefghijklmnopqrstuvwxyz')
    print(alphabet_list)
    with open(longtext_path) as fp:
        for line in fp:
            data = list(line.strip())
            for i in range(len(data)-1):
                alpha_i = data[i].lower()
                alpha_j = data[i+1].lower()
                if alpha_i not in alphabet_list and alpha_i != " ":
                    alpha_i = " "
                if alpha_j not in alphabet_list and alpha_j != " ":
                    alpha_j = " "
                key = alpha_i+alpha_j
                if key in scoring_params:
                    scoring_params[key]+=1
                else:
                    scoring_params[key]=1
    return scoring_params
scoring_params = create_scoring_params_dict('warandpeace.txt')
#print(scoring_params)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [41]:
ctext = 'sopxrqprphastzopfinbpbrupscprmasnprcbpoz pfnifjqpx a pqoasjsctpozsaoc'
key = ['r', 'k', 'f', 'b', ' ', 'h', 't', 'z', 's', 'g', 'j', 'n', 'd', 'c', 'i', 'm', 'y', 'a', 'q', 'o', 'e', 'v', 'x', 'w', 'u', 'l', 'p']
table = str.maketrans("".join(key), 'abcdefghijklmnopqrstuvwxyz ')
"".join(key)

'rkfb htzsgjndcimyaqoevxwulp'